### Создание графа Tracking network

In [1]:
import numpy as np
import re
from time import sleep
import chess 
import chess.pgn
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def algebraic_to_coordinate(algebraic):
    letter, number = algebraic[0], algebraic[1]
    x = ord(letter) - ord('a') + 1
    y = int(number)
    return (x, y)


def coordinate_to_algebraic(coordinate):
    x, y = coordinate
    letter = chr(ord('a') + x - 1)
    number = str(y)
    return letter + number


def intermediate_cells(start, end):
    start_coord = algebraic_to_coordinate(start)
    end_coord = algebraic_to_coordinate(end)
    
    x1, y1 = start_coord
    x2, y2 = end_coord
    
    intermediate = []
    
    x_step = 1 if x1 < x2 else -1 if x1 > x2 else 0
    y_step = 1 if y1 < y2 else -1 if y1 > y2 else 0

    if x_step == 0 or y_step == 0:
        steps = max(abs(x2 - x1), abs(y2 - y1))
    else:
        steps = abs(x2 - x1)

    x, y = x1, y1

    for _ in range(1, steps):
        x += x_step
        y += y_step
        intermediate.append(coordinate_to_algebraic((x, y)))
    
    return intermediate

In [ ]:
def create_tracking_network(input_file_path):
    with open(input_file_path) as pgn_file:
        while True:
            game = chess.pgn.read_game(pgn_file)
            board = chess.Board()
            G = nx.DiGraph()
            plt.figure(figsize=(10, 10))
            for number, move in enumerate(game.mainline_moves()): 
                board.push(move)
                array_of_cells = intermediate_cells(move.uci()[:2], move.uci()[2:])
                for i in range(len(array_of_cells) - 1):
                    G.add_edge(array_of_cells[i], array_of_cells[i + 1], color="black")
            pos = nx.spring_layout(G, k = 2)
            edges = G.edges(data=True)
            colors = [data['color'] for u, v, data in edges]
            nx.draw_networkx(G, pos, with_labels=True, edge_color=colors, node_size=1000, font_size=8, font_color='black', font_weight='bold')